# **Create NMT FS Surfaces**  

## Initiate 
Setting a few environment variables here so we can actually run code from this notebook

In [ ]:
SUBJ=NMT  # Name of the animal you are going to process
NMT_path=/NHP_MRI/Template/NMT # path to the template folder
startpath=pwd # current directory

#### Create folders & copy files

In [ ]:
# create a folder for the freesurfer surfaces we will eventually end up with
fsSurf=${NMT_path}/fsSurf
mkdir -p ${fsSurf}

# make subfolders for structure
fsSurf_src=${fsSurf}/src
fsSurf_mgz=${fsSurf}/mgz
fsSurf_temp=${fsSurf}/temp
mkdir -p ${fsSurf_src}
mkdir -p ${fsSurf_mgz}
mkdir -p ${fsSurf_temp}

# initialize an array to be able to loop over hemispheres
HEMI=(lh rh) 

In [ ]:
# copy the source files from the NMT segmentation to the freesurfer folders you just created
cp ${NMT_path}/NMT.nii.gz ${fsSurf_src}/T1.nii.gz
cp ${NMT_path}/NMT_SS.nii.gz ${fsSurf_src}/brain.nii.gz
cp ${NMT_path}/NMT_SS.nii.gz ${fsSurf_src}/brainmask.nii.gz # freesurfer brainmask aren't actually masks
cp ${NMT_path}/masks/probabilisitic_segmentation_masks/NMT_segmentation_WM.nii.gz ${fsSurf_src}/wm.nii.gz
cp ${NMT_path}/masks/anatomical_masks/NMT_blood_vasculature_mask.nii.gz ${fsSurf_src}/blood.nii.gz

# copy source files again but keep these in original state 
# (for others the header will be changed to 'fake' 1 mm voxels, which is necessary for freesurfer)
mkdir -p ${fsSurf_src}/org
cp ${fsSurf_src}/*.nii.gz ${fsSurf_src}/org/

#### Change headers to mimic 0.5 mm isotropic voxels & make mgz files
Freesurfer will downsample to 1 mm which should be ok

In [ ]:
# change headers
voxsz=0.5
3drefit -xdel $voxsz -ydel $voxsz -zdel $voxsz -keepcen ${fsSurf_src}/T1.nii.gz
3drefit -xdel $voxsz -ydel $voxsz -zdel $voxsz -keepcen ${fsSurf_src}/brain.nii.gz
3drefit -xdel $voxsz -ydel $voxsz -zdel $voxsz -keepcen ${fsSurf_src}/brainmask.nii.gz
3drefit -xdel $voxsz -ydel $voxsz -zdel $voxsz -keepcen ${fsSurf_src}/wm.nii.gz
3drefit -xdel $voxsz -ydel $voxsz -zdel $voxsz -keepcen ${fsSurf_src}/blood.nii.gz

In [ ]:
fslmaths ${fsSurf_src}/wm.nii.gz -sub ${fsSurf_src}/blood.nii.gz -thr 0.1 -bin ${fsSurf_src}/wm.nii.gz
fslmaths ${fsSurf_src}/wm.nii.gz -mul 255 ${fsSurf_src}/wm255.nii.gz
fslmaths ${fsSurf_src}/brain.nii.gz -mas ${fsSurf_src}/blood.nii.gz ${fsSurf_src}/subtractblood.nii.gz
fslmaths ${fsSurf_src}/brain.nii.gz -sub ${fsSurf_src}/subtractblood.nii.gz ${fsSurf_src}/brain.nii.gz
fslmaths ${fsSurf_src}/T1.nii.gz -sub ${fsSurf_src}/subtractblood.nii.gz ${fsSurf_src}/T1.nii.gz

In [ ]:
# convert to mgz files for freesurfer
mri_convert -c ${fsSurf_src}/T1.nii.gz ${fsSurf_mgz}/T1.mgz
mri_convert -c ${fsSurf_src}/brain.nii.gz ${fsSurf_mgz}/brain.mgz
mri_convert -c ${fsSurf_src}/brain.nii.gz ${fsSurf_mgz}/brainmask.mgz
mri_convert -c ${fsSurf_src}/brainmask.nii.gz ${fsSurf_mgz}/brainmask_binary.mgz -rt nearest
mri_convert -c ${fsSurf_src}/wm.nii.gz ${fsSurf_mgz}/wm.mgz -rt nearest
mri_convert -c ${fsSurf_src}/wm255.nii.gz ${fsSurf_mgz}/wm255.mgz -rt nearest

In [ ]:
# create brain.finalsurfs.mgz
mri_mask -T 5 ${fsSurf_mgz}/brain.mgz ${fsSurf_mgz}/brainmask.mgz ${fsSurf_mgz}/brain.finalsurfs.mgz

#### Get corpus callosum and pons voxel coordinates
These will later define the cutting planes for the surface generation so make sure that 1) the corpus callosum is in the middle, and 2) the pons coordinates are low enough to not cut through cortex. 

Corpus callosum:  
![CC](pics/CC_coordinates.png)

Pons: 
![PONS](pics/PONS_coordinates.png)

In [ ]:
# Inspect volume to get voxel coordinates
freeview -v ${fsSurf_mgz}/brain.mgz &

In [ ]:
cp ${fsSurf_mgz}/wm255.mgz ${fsSurf_mgz}/wm.mgz

In [ ]:
CC=(128 117 130) # corpus callosum
PONS=(128 152 108) # pons

#### Fill the white matter volume & copy for backup before fixing

In [ ]:
# Fill WM
mri_fill -CV ${CC[0]} ${CC[1]} ${CC[2]} \
    -PV ${PONS[0]} ${PONS[1]} ${PONS[2]} \
    ${fsSurf_mgz}/wm.mgz ${fsSurf_mgz}/filled.mgz
# copy the original white matter before applying fixes    
cp ${fsSurf_mgz}/wm.mgz ${fsSurf_mgz}/wm_nofix.mgz

#### Tesselate volumes & fix topology (First run)

In [ ]:
# Tesselate
# left hemisphere
mri_pretess ${fsSurf_mgz}/filled.mgz 255 ${fsSurf_mgz}/brain.mgz ${fsSurf_mgz}/wm_filled-pretess255.mgz
mri_tessellate ${fsSurf_mgz}/wm_filled-pretess255.mgz 255 ${fsSurf_temp}/lh.orig.nofix
# right hemisphere
mri_pretess ${fsSurf_mgz}/filled.mgz 127 ${fsSurf_mgz}/brain.mgz ${fsSurf_mgz}/wm_filled-pretess127.mgz
mri_tessellate ${fsSurf_mgz}/wm_filled-pretess127.mgz 127 ${fsSurf_temp}/rh.orig.nofix

# for both hemispheres
for xh in ${HEMI[@]}; do
    # create a version we can edit
    cp ${fsSurf_temp}/${xh}.orig.nofix ${fsSurf_temp}/${xh}.orig

    # post-process tesselation
    mris_extract_main_component ${fsSurf_temp}/${xh}.orig.nofix ${fsSurf_temp}/${xh}.orig.nofix
    mris_smooth -nw ${fsSurf_temp}/${xh}.orig.nofix ${fsSurf_temp}/${xh}.smoothwm.nofix
    mris_inflate ${fsSurf_temp}/${xh}.smoothwm.nofix ${fsSurf_temp}/${xh}.inflated.nofix
    mris_sphere -q ${fsSurf_temp}/${xh}.inflated.nofix ${fsSurf_temp}/${xh}.qsphere.nofix
    cp ${fsSurf_temp}/${xh}.inflated.nofix ${fsSurf_temp}/${xh}.inflated

    # fix topology
    mris_euler_number ${fsSurf_temp}/${xh}.orig
    mris_remove_intersection ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.orig
    mris_smooth -nw ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.smoothwm
    mris_inflate ${fsSurf_temp}/${xh}.smoothwm ${fsSurf_temp}/${xh}.inflated
done

#### Iterate manual adjustments and doing tesselation & topology fix again
Should look something like this. Keep going (adjust `wm.mgz` using the recon-edit function of Freeview) until you are happy with the result.

![LH_wm_inflated_lateral](pics/LH_wm_inflated_lateral.png) ![LH_wm_inflated_medial](pics/LH_wm_inflated_medial.png)


In [ ]:
# look at the result and apply fixes in WM definition
# you will want to get rid of weird 'stalks' and 'bridges'
freeview -v ${fsSurf_mgz}/brain.mgz -v ${fsSurf_mgz}/wm.mgz \
    -f ${fsSurf_temp}/lh.smoothwm ${fsSurf_temp}/lh.inflated \
    ${fsSurf_temp}/rh.smoothwm ${fsSurf_temp}/rh.inflated &

In [ ]:
# redo the tesselation with the freshly fixed WM volume
mri_fill -CV ${CC[0]} ${CC[1]} ${CC[2]} \
    -PV ${PONS[0]} ${PONS[1]} ${PONS[2]} \
    ${fsSurf_mgz}/wm.mgz ${fsSurf_mgz}/filled.mgz

The Pons-based cut may slice through the temporal lobes. Haven't found a way to fix this other than manually editing the `filled.mgz volume` after the `mri_fill` step.

In [ ]:
mri_pretess ${fsSurf_mgz}/filled.mgz 255 ${fsSurf_mgz}/brain.mgz ${fsSurf_mgz}/wm_filled-pretess255.mgz
mri_tessellate ${fsSurf_mgz}/wm_filled-pretess255.mgz 255 ${fsSurf_temp}/lh.orig
mri_pretess ${fsSurf_mgz}/filled.mgz 127 ${fsSurf_mgz}/brain.mgz ${fsSurf_mgz}/wm_filled-pretess127.mgz
mri_tessellate ${fsSurf_mgz}/wm_filled-pretess127.mgz 127 ${fsSurf_temp}/rh.orig

In [ ]:
for xh in ${HEMI[@]}; do
    mris_extract_main_component ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.orig
    mris_smooth -nw ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.smoothwm
    mris_inflate ${fsSurf_temp}/${xh}.smoothwm ${fsSurf_temp}/${xh}.inflated
    mris_sphere -q ${fsSurf_temp}/${xh}.inflated ${fsSurf_temp}/${xh}.qsphere
    
    mris_euler_number ${fsSurf_temp}/${xh}.orig
    mris_remove_intersection ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.orig
    mris_smooth -nw ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.smoothwm
    mris_inflate ${fsSurf_temp}/${xh}.smoothwm ${fsSurf_temp}/${xh}.inflated
    mris_curvature -thresh .999 -n -a 5 -w -distances 10 10 ${fsSurf_temp}/${xh}.inflated
done

In [ ]:
# create the spheres (this takes longer so do it only when you're happy with the inflated)
for xh in ${HEMI[@]}; do
    mris_sphere ${fsSurf_temp}/${xh}.inflated ${fsSurf_temp}/${xh}.sphere
done

#### Create a subject directory in your freesurfer subject directory

In [ ]:
# create a freesurfer SUBJECT with the correct folder structure
# NB! $SUBJECTS_DIR should be defined in your ~/.bashrc as the Freesurfer subjects directory
echo 'Creating a subject directory for '${SUBJ} 'in:'
echo ${SUBJECTS_DIR}/${SUBJ}
mksubjdirs ${SUBJECTS_DIR}/${SUBJ}

#### Move files to the Freesurfer subjects directory

In [ ]:
# copy the necessary files for cutting to the subject directory
for xh in ${HEMI[@]}; do
    cp ${fsSurf_temp}/${xh}.inflated ${SUBJECTS_DIR}/${SUBJ}/surf/
    cp ${fsSurf_temp}/${xh}.smoothwm ${SUBJECTS_DIR}/${SUBJ}/surf/
    cp ${fsSurf_temp}/${xh}.orig ${SUBJECTS_DIR}/${SUBJ}/surf/
    cp ${fsSurf_temp}/${xh}.qsphere ${SUBJECTS_DIR}/${SUBJ}/surf/
done

cp ${fsSurf_mgz}/T1.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/T1.mgz
cp ${fsSurf_mgz}/filled.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/filled.mgz
cp ${fsSurf_mgz}/wm.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/wm.mgz
cp ${fsSurf_mgz}/brain.finalsurfs.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/brain.finalsurfs.mgz
cp ${fsSurf_mgz}/brainmask.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/brainmask.mgz

Before we can start making the final surfaces we need a little trick to compensate for the low grey/white matter contrast in monkeys. Now that we have a white matter volume we are happy with (if not go back and fix it) we can use it to enhance the GM/WM contrast artificially and bring the voxel values into the range that Freesurfer expects. The difference looks somewhat like this (Left: normal brain, Right: contrast enhanced brain).

![Brain](pics/brain.png) ![Brain_enh](pics/brain_enhanced.png)

#### Make a contrast-enhanced brain volume to aid in getting the pial surface

In [ ]:
# Make a folder for this procedure and convert the necessary files to nifti so we can use `fslmaths`
mkdir -p $fsSurf/enh
mri_convert ${fsSurf_mgz}/brain.mgz ${fsSurf}/enh/brain.nii.gz 
mri_convert ${fsSurf_mgz}/wm.mgz ${fsSurf}/enh/wm.nii.gz 

# binarize the wm volume
fslmaths ${fsSurf}/enh/wm.nii.gz -thr 10 -bin ${fsSurf}/enh/wm_bin.nii.gz

In [ ]:
# divide the voxel value in the original brain volume by **some** value and add **another** value to the white matter only 
# which values you need here is up to you but we want to end with white matter voxels having a value ~110 and grey matter ~60-80
# alternative ways are definitely possible as long as your end result is similar
#fslmaths ${fsSurf}/enh/brain.nii.gz -div 2 ${fsSurf}/enh/brain2.nii.gz
#fslmaths ${fsSurf}/enh/wm_bin.nii.gz -mul 20 ${fsSurf}/enh/wm_add.nii.gz
#fslmaths ${fsSurf}/enh/brain2.nii.gz -add ${fsSurf}/enh/wm_add.nii.gz ${fsSurf}/enh/brain_enh.nii.gz

fslmaths ${fsSurf}/enh/brain.nii.gz -div 2 ${fsSurf}/enh/brain_enh.nii.gz


# convert the 'enhanced' brain back to mgz
mri_convert ${fsSurf}/enh/brain_enh.nii.gz ${SUBJECTS_DIR}/${SUBJ}/mri/brain_enh.mgz

#### Create the surfaces

In [ ]:
# create surfaces
# this will take a while. don't wait for it. go get a coffee, call your mother, or read a paper.
for xh in ${HEMI[@]}; do
    mris_make_surfaces -noaseg -noaparc -T1 brain_enh -orig_wm orig ${SUBJ} ${xh}
    mris_sphere ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.inflated ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.sphere
done

You should now have nicely looking inflated white matter surfaces and a projection of it onto a sphere surface. They should look somewhat like the following images.
`freeview -f ${fsSurf_temp}/lh.smoothwm ${fsSurf_temp}/lh.inflated ${fsSurf_temp}/lh.sphere &`

![surfaces_leged](pics/final_surf_legend.png)

In [ ]:
# freeview -f ${fsSurf_temp}/lh.smoothwm ${fsSurf_temp}/lh.inflated ${fsSurf_temp}/lh.sphere &
freeview -f ${SUBJECTS_DIR}/${SUBJ}/surf/lh.smoothwm ${SUBJECTS_DIR}/${SUBJ}/surf/lh.inflated ${SUBJECTS_DIR}/${SUBJ}/surf/lh.sphere &

In [ ]:
# check and edit the pial surface
mkdir -p ${SUBJECTS_DIR}/${SUBJ}/pial_edits # create a folder for pial edits
cp -R ${SUBJECTS_DIR}/${SUBJ}/mri ${SUBJECTS_DIR}/${SUBJ}/pial_edits/ 
cp -R ${SUBJECTS_DIR}/${SUBJ}/surf ${SUBJECTS_DIR}/${SUBJ}/pial_edits/

In [ ]:
freeview -v ${SUBJECTS_DIR}/${SUBJ}/pial_edits/mri/T1.mgz ${SUBJECTS_DIR}/${SUBJ}/pial_edits/mri/brain_enh.mgz \
    -f ${SUBJECTS_DIR}/${SUBJ}/pial_edits/surf/lh.white:edgecolor=yellow ${SUBJECTS_DIR}/${SUBJ}/pial_edits/surf/lh.pial:edgecolor=red \
    ${SUBJECTS_DIR}/${SUBJ}/pial_edits/surf/rh.white:edgecolor=yellow ${SUBJECTS_DIR}/${SUBJ}/pial_edits/surf/rh.pial:edgecolor=red &

In [ ]:
# NB! If you adjustments to the brainmask (which probably isn't necessary if NMT segmentation worked well) 
# don't forget to copy back the adjusted brain_enh for re-generation of the pial surface 
cp ${SUBJECTS_DIR}/${SUBJ}/pial_edits/mri/brain_enh.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/brain_enh.mgz

In [ ]:
# regenerate the pial surface
for xh in ${HEMI[@]}; do
    mris_make_surfaces  -noaseg -noaparc -orig_white white -orig_pial white -nowhite -mgz -T1 brain_enh ${SUBJ} ${xh}
done

In [ ]:
# create midcortical surface by growing the white matter halfway towards the pial surface
for xh in ${HEMI[@]}; do
    mris_expand -thickness ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.white 0.5 ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.graymid
done

In [ ]:
# Fix topology, smooth & inflate the final surfaces
for xh in ${HEMI[@]}; do    
    mris_euler_number ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.white
    mris_remove_intersection ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.white ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.white
    mris_smooth -n 3 -nw ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.white ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.smoothwm
    
    mris_euler_number ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.pial
    mris_remove_intersection ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.pial ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.pial
    mris_smooth -n 3 -nw ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.pial ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.smoothpial
    
    mris_euler_number ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.graymid
    mris_remove_intersection ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.graymid ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.graymid
    mris_smooth -n 3 -nw ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.graymid ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.smoothgraymid
    
    mris_inflate ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.smoothwm ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.inflated
    mris_curvature -thresh .999 -n -a 5 -w -distances 10 10 ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.inflated
    mris_sphere ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.inflated ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.sphere
done

#### Cut surfaces and create flatmaps

NB! Freesurfer's `tktools` do not work on Mac OS X 10.15 (Catalina) and later because they are 32 bits. This step could be run on a (virtual) machine with a different OS. Alternatively, cuts can perhaps be made with other software. The [pycortex](https://github.com/gallantlab/pycortex) documentation mentions that it can be done with [Blender](https://www.blender.org/), but I have not tested this yet.

The cells below will load the inflated hemispheres in tksurfer where you can make the cuts to create the flatmaps.  

For the full hemisphere make cuts on the medial wall. One cut enclosing the midline and five additional relaxation cuts.
Mark a point on the part of the surface you want to keep and save patch as `?h.full.patch.3d` (replace '?' with 'l' and 'r' respectively)

![Cut_full](pics/Cut_full.png)

For an occipital patch make one cut on the medial wall along the calcarine sulcus. Use 3 points to select a coronal cutting plane, and a fourth point to select which part of the surface you want to keep and save as `?h.occip.patch.3d`

![Cut_occip1](pics/Cut_occip1.png) ![Cut_occip1](pics/Cut_occip2.png)

In [ ]:
# left
#tksurfer ${SUBJ} lh inflated -curv
tksurfer ${SUBJ} lh inflated -gray 

In [ ]:
# right
#tksurfer ${SUBJ} rh inflated -curv
tksurfer ${SUBJ} rh inflated -gray 

In [ ]:
# Time for another coffee, you are not going to want to wait for this to finish while staring at its progress
cd ${SUBJECTS_DIR}/${SUBJ}/surf/
for xh in ${HEMI[@]}; do
    mris_flatten -w 0 -distances 20 7 ${xh}.full.patch.3d  ${xh}.full.patch.flat
    mris_flatten -w 0 -distances 20 7 ${xh}.occip.patch.3d  ${xh}.occip.patch.flat
done

If everything went well, you now have flatmaps of the white matter surface. 
They should look somewhat like this:  

**Full hemisphere (rh)**  
![RH_flatmap](pics/RH_Flat_Sulci_gray_nolegend.png) ![RH_flatmap_legend](pics/RH_Flat_Sulci_gray_legend.png)

**Occipital (rh)** (functional areas are guesstimates based on anatomy)  
![RH_occip](pics/RH_Flat_occip_gray_nolegend.png) ![RH_occip_gray_legend](pics/RH_Flat_occip_gray_legend.png)

It is of course also possible to make **any** other funtionally relevant cutting pattern to extract a flatmap of a piece of cortex. The method is the same.

In [ ]:
# check your result
xh=lh
#tksurfer ${SUBJ} rh inflated -patch rh.full.patch.flat -curv # for a red/green curvature map
tksurfer ${SUBJ} ${xh} inflated -patch ${xh}.full.patch.flat -gray # for a gray curvature map

#### Convert Freesurfer surfaces to Gifti
Other software may like this better.

In [ ]:
mkdir -p gii
for xh in ${HEMI[@]}; do
    mris_convert ${xh}.white ./gii/${xh}.white.gii
    mris_convert ${xh}.graymid ./gii/${xh}.graymid.gii
    mris_convert ${xh}.pial ./gii/${xh}.pial.gii
    mris_convert ${xh}.inflated ./gii/${xh}.inflated.gii

    mris_convert ${xh}.smoothwm ./gii/${xh}.smoothwm.gii
    mris_convert ${xh}.smoothgraymid ./gii/${xh}.smoothgraymid.gii
    mris_convert ${xh}.smoothpial ./gii/${xh}.smoothpial.gii
    
    mris_convert -p -c ${xh}.curv ${xh}.full.patch.flat ./gii/${xh}.full.patch.flat.gii
    mris_convert -p -c ${xh}.curv ${xh}.occip.patch.flat ./gii/${xh}.occip.patch.flat.gii
done

#### Copy Freesurfer results back to NMT template folder

In [ ]:
cp -r ${SUBJECTS_DIR}/${SUBJ}/surf ${fsSurf}/surf

In [ ]:
# return to startpath
cd ${startpath}